In [ ]:
import numpy as np
import pandas as pd
import pickle
from glob import glob
import re
# import nltk
# import unidecode

with open('NB_Model_english_char_ngram4', 'rb') as files:
    model_eng = pickle.load(files)
    
with open('tfidf_vectorizer_english_char_ngram4', 'rb') as files:
    tfidf_eng = pickle.load(files)    

with open('Label_Encoder_english_char_ngram4', 'rb') as files:
    le_eng = pickle.load(files)
    

with open('NB_Model', 'rb') as files:
    model_ind = pickle.load(files)
    
with open('tfidf_vectorizer', 'rb') as files:
    tfidf_ind = pickle.load(files)    

with open('Label_Encoder', 'rb') as files:
    le_ind = pickle.load(files)    

In [ ]:
le_ind.inverse_transform(model_ind.classes_)

In [116]:
vocab = open('eng_vocab.txt')
voc = [str(word.replace('\n', '')) for word in vocab.readlines()]

In [117]:
voc = [word for word in voc if word!='are' and word!='will' and word!='bill' and word!='a' and word!='me'
      and word!='hi' and word!='main' and word!='may' and word!='by' and word!='or' and word!='per'
      and word!='pan' and word!='sir' and word!='us' and word!='it' and word!='do']

In [118]:
vocab.close()

In [119]:
if 'do' in voc:
    print('Y')

In [ ]:
# vector = tfidf_eng.transform(['do']).toarray() 
# language = model_eng.predict(vector) 
# language_pct = model_eng.predict_proba(vector) 
# language = le_eng.inverse_transform(language)
# print(f'''{str(language[0])}: {np.max(language_pct)}''') 

In [23]:
def predictFind_codemix(text): 
    
    global voc, model_eng, tfidf_eng, le_eng, model_ind, tfidf_ind, le_ind 
    text = re.sub(r'[!@#$(),"%^*.?:;~`0-9]', ' ', str(text))
    text = re.sub(r'[<>]', ' ', str(text))
    text = re.sub(r'[~={}]', ' ', str(text))
    text = re.sub(r'[+-]', ' ', str(text))    
    text = re.sub(r'[[]]', ' ', str(text))
    text = re.sub(r' +', ' ', str(text))
    text = text.lower()
    text = text.strip()    
    english_words = []
    text_for_pred = []
    
    for token in text.split():
        vector = tfidf_eng.transform([token]).toarray() 
        language = model_eng.predict(vector) 
        language_pct = model_eng.predict_proba(vector) 
        language = le_eng.inverse_transform(language)
        if token in voc or ( str(language[0])=='English' and np.max(language_pct)>0.09 ):
            english_words.append(token)
    
    text_for_pred = ' '.join([token for token in text.split() if token not in english_words])
    
    x = tfidf_ind.transform([text_for_pred]).toarray() 
    lang = model_ind.predict(x) 
    lang_per = model_ind.predict_proba(x) 
    lang = le_ind.inverse_transform(lang)
    print(f'''The text is in {str(lang[0])}\n\n''') 
    
#     lang_names = list(le.inverse_transform(model.classes_)) 
#     probas_list = []
#     for i in lang_per:
#         for j in i:
#             probas_list.append(j)

#     proba_table = pd.DataFrame({'Language': lang_names,
#                         'Probability': probas_list}).sort_values(by='Probability',
#                                                            ascending=False).reset_index(drop=True)

    if len(english_words)>0 and len(english_words)/len(text.split())<0.60:
        with open(f'''{str(lang[0])}-English-codemix.csv''', 'a') as file:
            file.write(f'{text}\n')      
            file.close()
    elif len(english_words)==0:
        pass

In [ ]:
files = glob('*.csv')

In [15]:
for index, file in enumerate(files):
    print(f'CSV file number: {index}')
    with open(file, 'r') as f:
        data = f.readlines()
    
    for text in data:
        predictFind_codemix(text)